In [3]:
""" Scrape CTIS for Adverse status of CRC contracts """

import requests as req
import bs4 as bs
import csv
import login

period = '20171'
trigger = 'N'

session = req.session()
session.headers.update({"User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36"})

url = 'http://bvca07416/index.php'
form = {'action': 'check', 'userid': login.ctis_username, 'pwd': login.ctis_password}
response = session.post(url, data=form, verify=False)

url = 'http://bvca07416/ctis_mass_crc_entry.php'
form={'sel_period': period, 'trigger': trigger}
response = session.post(url, data=form, verify=False)

with open('./output/' + 'ctis_mass_crc_entry_' + period + '_' + 'all.html' if trigger == 'N' else './' + 'ctis_mass_crc_entry_' + period + '_' + 'triggered.html', 'wb') as f:
    f.write(response.content)
    
soup = bs.BeautifulSoup(response.content, 'lxml')
table = soup.find_all('table')[2]
rows = table.find_all('tr')[2:-2]

header = ['contract', 'contract_title', 'contractor', 'contract_remark', 'cm_recommendation', 'crc_rating', 'warning_letter', 'interview', 'removed_from_qmc', 'wef', 'chairperson_recommendation', 'other_remark', 'follow_up_action']

with open('./output/' + 'ctis_mass_crc_entry_' + period + '_' + 'all.csv' if trigger == 'N' else './' + 'ctis_mass_crc_entry_' + period + '_' + 'triggered.csv', "w", newline='') as csv_file:
    writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    writer.writerow(header)
    for row in rows:
        cols = row.find_all('td')
        tr = []
        tr.extend([cols[0].text.strip(), cols[1].text, cols[2].text, cols[3].textarea.text])
        tr.extend([cols[4].find_all('option', {'selected': ['']})[0].text])
        tr.extend([cols[5].find_all('option', {'selected': ['']})[0].text])
        tr.extend(['N' if len(cols[6].find_all('checked')) == 0 else 'Y'])
        tr.extend(['N' if len(cols[7].find_all('checked')) == 0 else 'Y'])
        tr.extend(['N' if len(cols[8].find_all('checked')) == 0 else 'Y'])
        tr.extend([cols[9].input['value']])
        tr.extend([cols[10].find_all('option', {'selected': ['']})[0].text])
        tr.extend([cols[11].textarea.text])
        tr.extend([cols[12].textarea.text])
        writer.writerow(tr)
